In [1]:
import pandas as pd
import numpy as np

## Here is the list of Dodgy Drugs:
Some of them had NA smiles.
Some of them had the same name but different smiles were reported for them
Some of them had the same name but different Inchikeys were reported for them
I have shown the number of replicates in dual therapy summary dataframe for them and sorted them based on the number of replicates

In [69]:
pd.set_option('display.max_rows', None)
df

,dname,replicates
7,dacarbazine,6195
2,teniposide,6161
34,Vincristine sulfate,5872
38,geldanamycin,5775
19,LSN415169,3763
4,SrcI1,3666
12,vasastrol,3659
29,MED-575,555
11,TRAIL,307
59,AZ13453720,303


In [6]:
druginfo = pd.read_csv(r"C:\PhD\2024\Data\Drugcomb\Venn_diagram_Contradiction_Completing_Smiles_from_drugbank_pubchem\druginfo_add_smiles_from_cid_and_drugbank.csv")
summ = summ = pd.read_csv(r"C:\PhD\2024\Data\Drugcomb\DrugComb_Summary_ReadyTo_Preprocessing.csv", low_memory=False)

## Only considers drugs available in dual therapy

In [47]:
summ.dropna(subset=["drug_col"], inplace=True)
summ.dropna(subset=["drug_row"], inplace=True)
sum_drug_list = list(set(summ["drug_col"]) | set(summ["drug_row"]))
druginfo = druginfo[druginfo["dname"].isin(sum_drug_list)]
druginfo["cid"] = druginfo["cid"].fillna(0).astype(int)
druginfo.dropna(subset=['dname'],inplace=True)

dodgy_drugs = {
    'null_smiles': set(),
    'same_drugs_name_different_smiles': set(),
    'same_drugs_name_different_inchikeys_0:13': set()
}


In [48]:
druginfo.head(2)

,dname,id,chembl_id,inchikey,smiles,cid,molecular_formula,clinical_phase,cid_m,cid_s,stitch_name,drugbank_id,kegg_id,synonyms,target_name,target_type,pubchem_cid_to_smiles,drugbank_id_to_smiles,inchikeys_from_drugbankID
0,5-Fluorouracil,1,CHEMBL185,GHASVSINZRGABV-UHFFFAOYSA-N,C1=C(C(=O)NC(=O)N1)F,3385,C4H3FN2O2,4.0,CIDm00003385,CIDs00003385,5-fluorouracil,DB00544,D00584,5-Fluorouracil; fluorouracil; 51-21-8; 5-FU; F...,Prelamin-A/C; Survival motor neuron protein; T...,single protein; nucleic acid,NaN,FC1=CNC(=O)NC1=O,NaN
1,Veliparib,2,CHEMBL506871,JNAHVYVRKWKWKQ-CYBMUJFWSA-N,CC1(CCCN1)C2=NC3=C(C=CC=C3N2)C(=O)N,11960529,C13H16N4O,3.0,CIDm11842604,CIDs11960529,veliparib,DB07232,D09692,Veliparib; 912444-00-9; ABT-888; ABT 888; ABT-...,Poly [ADP-ribose] polymerase-1; Poly [ADP-ribo...,single protein; protein family,C[C@@]1(CCCN1)C2=NC3=C(C=CC=C3N2)C(=O)N,C[C@@]1(CCCN1)C1=NC2=CC=CC(C(N)=O)=C2N1,NaN


# Duplicate Drug names

In [71]:
def duplicate(based, druginfo):

    duplicates = druginfo[druginfo[based].duplicated(keep=False)]
    
    duplicates_sorted = duplicates.sort_values(by=based)
    l = len(duplicates_sorted[based].unique())
    print(f"Number of Drugs having multipe samples but one {based}: {l}")

    for sample in duplicates_sorted[based].unique():
        results_rep = []
        results_unique =[]
        #print(f"number of samples having same {based} having value of {sample}: {len(duplicates_sorted[duplicates_sorted[based]==sample])}")
        dupd = duplicates_sorted[duplicates_sorted[based] == sample]
        #items_to_check = ['dname','chembl_id','inchikey','smiles','cid','molecular_formula','clinical_phase','cid_m','cid_s','stitch_name','drugbank_id','kegg_id','synonyms','target_name','target_type','pubchem_cid_to_smiles','drugbank_id_to_smiles','inchikeys_from_drugbankID']
        items_to_check = ['inchikey','smiles']
        for item in  items_to_check:
            #print("_____________________")
            values = dupd[item].unique()
            if item == 'inchikey':
                # Consider only the first 14 characters for each InChIKey
                values = [value[:14] for value in values if isinstance(value, str)]
                values = list(set(values))

            if len(values)==1:
                #print(f"{sample} have unique value {item} although the {based} is duplicated")
                results_unique.append((sample, values[0]))

            else:
                if item == 'smiles':
                    dodgy_drugs['same_drugs_name_different_smiles'].add(sample)

                if item == 'smiles':
                    dodgy_drugs['same_drugs_name_different_inchikeys_0:13'].add(sample)                
                results_rep.append((sample, values))
                #print(f"{sample} have {len(values)} distinct {item} while {based} is duplicated")
                #print(values)
    
        #print("***********************************************")

duplicate('dname',druginfo)

Number of Drugs having multipe samples but one dname: 94


# Null Smiles
All other information for these drugs were NaN so I add these into my remove list.

In [50]:
print(f"Number of drugs having missing value in smiles: {len(druginfo[pd.isna(druginfo['smiles'])])}")
smilesna = druginfo[pd.isna(druginfo['smiles'])]
setdrugn = set(smilesna['dname'])
dodgy_drugs['null_smiles'].update(setdrugn)

Number of drugs having missing value in smiles: 19


In [56]:
unique_drugs = set()
for value_set in dodgy_drugs.values():
    unique_drugs.update(value_set)

unique_drug_list = list(unique_drugs)

print(f"number of dodgy drugs: {len(unique_drug_list)}")


number of dodgy drugs: 66


# The list of the dodgy drugs

In [ ]:
df = pd.DataFrame(columns =['dname','replicates'])
df['dname']=unique_drug_list
for i, row in df.iterrows():
    dname = row['dname']
    df.loc[i,'replicates']=len(summ[summ['drug_row']==dname])+len(summ[summ['drug_col']==dname])

df = df.sort_values(by='replicates',  ascending=False)


In [ ]:
df